In [2]:
import os
from pathlib import Path 
import sys
sys.path.append('../../dataset')
sys.path.append("/home/kirilman/Project/asbestos/JSON2YOLO")
from utils import get_paths_from_dirs
from general_json2yolo import convert_coco_json
import shutil
import json
import numpy as np

In [3]:
datasets_names = ['pits_stones_detections_020221',
                  'pits_stones_detections_161120',
                  'pits_stones_detections_111121',
                  'pits_stones_detections_161220',
                  'pits_stones_detections_300920',
                  'asbestos_stone_020221',
                  'open-pit-zoomed-161120',
                  'asbest_stones_111121',
                  'test transport',
                  'asbestos_stone_161220']

In [14]:
def is_file_image(p:Path):
    return p.suffix.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))

In [281]:
root_to_dataset = Path('/home/kirilman/Project/dataset')
images_paths = []
for data_set in datasets_names:
    p = (root_to_dataset / data_set).resolve()
    r = Path(p).resolve().rglob('*.*')
    for e in r:
        if is_file_image(e):
            images_paths.append(e)
            
len(images_paths)

394

### Скопировать изображения

In [247]:
images_files = []
for data_set in datasets_names:
    p = (root_to_dataset / data_set).resolve()
    r = get_paths_from_dirs([p], ['*'])
    for e in r:
        if is_file_image(e):
            images_files+=[e]

path_2_detection_set = Path('/home/kirilman/Project/dataset/detection_set2/images')
if path_2_detection_set.exists():
    shutil.rmtree(path_2_detection_set)
    path_2_detection_set.mkdir(parents=True,exist_ok=True)
else:
    path_2_detection_set.mkdir(parents=True,exist_ok=True)

#скопировать
for p in images_files:
    shutil.copyfile(p, path_2_detection_set / p.name)

### Скопировать json файлы с разметкой

In [242]:
labels_files = []
for data_set in datasets_names:
    p = (root_to_dataset / data_set).resolve()
    r = get_paths_from_dirs([p], ['json'])
    for e in r:
        labels_files+=[e]

path_2_labels = Path('/home/kirilman/Project/dataset/detection_set2/labels')
if path_2_labels.exists():
    shutil.rmtree(path_2_labels)
    path_2_labels.mkdir(parents=True,exist_ok=True)
else:
    path_2_labels.mkdir(parents=True,exist_ok=True)

#скопировать
for p in labels_files:
    shutil.copyfile(p, path_2_labels / (p.parent.parts[-2] + p.suffix))

In [267]:
#фикс путь до изображений
for label_file in get_paths_from_dirs([path_2_labels], ['json']):
    f = open(label_file)
    data = json.load(f)
    f.close()
    
    for i, im in enumerate(data['images']):
        p = Path(im['file_name'])
        data['images'][i]['file_name'] = p.name
        
        with open(f.name, 'w') as outfile:
            json.dump(data, outfile)

In [277]:
### Convert to yolo

In [315]:
path_2_labels = '/home/kirilman/Project/dataset/detection_set2/coco_labels/' 
json_files = get_paths_from_dirs([path_2_labels], ['json'])
path_to_yolo = '/home/kirilman/Project/dataset/detection_set2/labels/'
convert_coco_json(json_files[0].parent, path_to_yolo)

Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/asbest_sto
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/asbestos_s
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/asbestos_s
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/open-pit-z
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/pits_stone
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/pits_stone
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/pits_stone
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/pits_stone
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/pits_stone
Annotations /home/kirilman/Project/dataset/detection_set2/coco_labels/test trans


In [316]:
json_files = get_paths_from_dirs([path_to_yolo], ['txt'])
len(json_files)
for f in json_files:
    shutil.copy(f, path_to_yolo)

In [11]:
json_files = get_paths_from_dirs([path_to_yolo], ['txt'])
images_files = get_paths_from_dirs([path_2_detection_set], ['*'])
print(len(json_files), len(images_files))

NameError: name 'path_to_yolo' is not defined

In [305]:
##DELETE Asbests bboxs

In [334]:
path_2_yolo = Path('/home/kirilman/Project/dataset/detection_set2/labels/rocks and asbest/')
yolo_files = os.listdir(path_2_yolo)
for name in yolo_files:
    outfile = open(path_2_yolo.parent / name, "w")
    file = open(path_2_yolo / name, 'r') 
    lines = file.readlines()
    for line in lines:
        if line in lines:
            if line[0] == "1":
                continue
            else:
                outfile.write(line)
    outfile.close()
    file.close()

### Разбиение на обучение тест

In [4]:
image_files = get_paths_from_dirs(['/home/kirilman/Project/dataset/detection_set2/images/'], ['*'])
label_files = get_paths_from_dirs(['/home/kirilman/Project/dataset/detection_set2/labels/'], ['*'])

print("Всего изображений: ", len(image_files))
N = len(image_files)
K = round(0.25 * N)

print("Тест + Валидация: ",K)
indexs = np.random.choice(range(N), K)

test_indexs = indexs[:-20]
val_indexs  = indexs[-20:]
print("Tecт         ",len(test_indexs))
print("Валидация    ", len(val_indexs))

Всего изображений:  394
Тест + Валидация:  98
Tecт          78
Валидация     20


In [5]:
def img2label_paths(img_paths):
    # Define label paths as a function of image paths
    sa, sb = f'{os.sep}images{os.sep}', f'{os.sep}labels{os.sep}'  # /images/, /labels/ substrings
    return [sb.join(x.rsplit(sa, 1)).rsplit('.', 1)[0] + '.txt' for x in img_paths]

In [44]:
path_2_images = image_files[0].parent
path_2_validation = path_2_images.parent / 'validation'
path_2_test       = path_2_images.parent / 'test'

if path_2_validation.exists():
    shutil.rmtree(path_2_validation)
    path_2_validation.mkdir(parents=True,exist_ok=True)
else:
    path_2_validation.mkdir(parents=True,exist_ok=True)
    
if path_2_test.exists():
    shutil.rmtree(path_2_test)
    path_2_test.mkdir(parents=True,exist_ok=True)
else:
    path_2_test.mkdir(parents=True,exist_ok=True)


In [45]:
#копируем файлы
#тестовые
for path_image in [p for i,p in enumerate(image_files) if i in test_indexs]:
    path_label = img2label_paths([str(path_image)])[0]
    shutil.copy(path_image, path_2_test / path_image.name)
    shutil.copy(path_label, path_2_test / Path(path_image.name.split('.')[0] + '.txt') )

#Валидация

for path_image in [p for i,p in enumerate(image_files) if i in val_indexs]:
    path_label = img2label_paths([str(path_image)])[0]
    shutil.copy(path_image, path_2_validation / path_image.name)
    shutil.copy(path_label, path_2_validation / Path(path_image.name.split('.')[0] + '.txt') )
